In [ ]:
from yuzu.exchanges.binanceus import BinanceUS
import json
exchange = BinanceUS(None, None)
order = exchange.create_order('buy', 'ADAUSD', test=True)
print(order)
balance = exchange.get_balances()
print(json.dumps(balance, indent=2))

In [ ]:
from optimize import optimize
from strategies.macdas_strat import macdas_strat, config_bounds
optimize(macdas_strat, config_bounds, 'ADAUSD', '1m', ticks=4000)

In [ ]:
from optimize import optimize
from strategies.macdas_strat import macdas_strat, config_bounds
optimize(macdas_strat, config_bounds, 'ADAUSD', '1h', ticks=4000)

In [ ]:
from yuzu.exchanges.binanceus import get_backdata
from strategies.macdas_strat import macdas_strat
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pandas import DataFrame
from numpy import full, nan
from flask import Flask

config = {'buy_slow_len': 35,
  'buy_fast_len': 23,
  'buy_sig_len': 9,
  'buy_rsi_len': 24,
  'sell_slow_len': 26,
  'sell_fast_len': 18,
  'sell_sig_len': 4,
  'sell_rsi_len': 13,
  'rsi_lb': 38.7,
  'rsi_ub': 75.1}

def bar(data, name):
    marker_colors = full(data[name].shape, nan, dtype=object)
    marker_colors[(data[name] > data[name].shift()) & (data[name] > 0)] = "green"
    marker_colors[(data[name] > data[name].shift()) & (data[name] < 0)] = "RGBA(255, 0, 0, .25)"
    marker_colors[data[name] == data[name].shift()] = "grey"
    marker_colors[(data[name] < data[name].shift()) & (data[name] > 0)] = "RGBA(0, 255, 0, .25)"
    marker_colors[(data[name] < data[name].shift()) & (data[name] < 0)] = "red"
    return go.Bar(y=data[name], x=data.index, name=name, marker_color=marker_colors)

def line(data, name, color):
    return go.Scatter(y=data[name], x=data.index, mode="lines", line_shape="spline", name=name, line=dict(color=color))

def plot(data):
    fig = make_subplots(rows=3, cols=1)
    fig.add_trace(go.Candlestick(x=data.index, open=data.open, high=data.high, low=data.low, close=data.close, name='ADAUSD'), row=1, col=1)
    fig.add_trace(go.Scatter(y=data.buy, x=data.index, name="buy", mode="markers", marker=dict(color="cyan", symbol="circle-open", size=10)), row=1, col=1)
    fig.add_trace(go.Scatter(y=data.sell, x=data.index, name="sell", mode="markers", marker=dict(color="yellow", symbol="circle-open", size=10)), row=1, col=1)
    fig.add_trace(bar(data, 'buy_hist'), row=2, col=1)
    fig.add_trace(line(data, 'buy_rsi', 'purple'), row=3, col=1)
    fig.update_xaxes(rangeslider_visible=False, spikemode="across", spikesnap="cursor", spikedash="dot", spikecolor="grey", spikethickness=1)
    fig.update_layout(template="plotly_dark", hovermode="x", spikedistance=-1)
    fig.update_traces(xaxis="x")
    fig.show()

data = macdas_strat(get_backdata('ADAUSD', '1h', 4000), config)
print(data)
plot(data)

In [ ]:
win_rate = lambda trades: len(list(filter(lambda t: t['win'], trades))) / len(trades)

In [ ]:
from itertools import groupby

def consectutive_win_rate(trades):
    consecs = [(n,len(list(c))) for n,c in groupby(trades,lambda t: t['win'])]
    most_consec_wins = max([t[1] for t in list(filter(lambda t: t[0], consecs))])
    most_consec_loses = max([t[1] for t in list(filter(lambda t: not t[0], consecs))])
    return most_consec_wins / (most_consec_wins + most_consec_loses)

In [ ]:
avg_profit = lambda trades: sum([t['close']-t['open'] for t in trades]) / len(trades)

In [ ]:
def avg_drawdown(trades):
    wins = list(filter(lambda t: t['win'], trades))
    drops = [(t['open']-t['low'])/t['open'] for t in trades]
    return sum(drops)/len(drops)

In [ ]:
def score(trades):
    return win_rate(trades) + consectutive_win_rate(trades) + avg_profit(trades) - avg_drawdown(trades)

In [ ]:
from numpy import isnan
from pandas import DataFrame

def quick_backtest(data: DataFrame, trading_fee=0.001) -> float:
    final_price = data.close.iat[-1]
    sub_data = data[['buy','sell']].copy(deep=False)
    sub_data = sub_data.dropna(subset=['buy','sell'], how='all')
    if not sub_data.empty:
        if not sub_data.sell.iat[0] is None:
            sub_data.loc[sub_data.index[0], "sell"] = None
        if not sub_data.buy.iat[-1] is None:
            sub_data.loc[sub_data.index[-1], "buy"] = None
    trades = []
    open_trade = False
    for index, tick in sub_data.iterrows():
        if not open_trade and not isnan(tick.buy):
            trades.append({'open': tick.buy, 'i': index})
            open_trade = True
        elif open_trade and not isnan(tick.sell):
            curr_trade = trades[-1]
            trades[-1] = {
                'open': curr_trade['open'],
                'close': tick.sell,
                'low': min(data.loc[curr_trade['i']:index, 'low']),
                'win': tick.sell > curr_trade['open']
            }
            open_trade = False
    return trades, score(trades)

In [ ]:
from yuzu.exchanges.BinanceUS import get_backdata
from yuzu.strategies.macdas_strat import macdas_strat

config = {
    'buy_slow_len': 32,
    'buy_fast_len': 17,
    'buy_sig_len': 4,
    'buy_rsi_len': 5,
    'sell_slow_len': 29,
    'sell_fast_len': 7,
    'sell_sig_len': 3,
    'sell_rsi_len': 31,
    'rsi_lb': 34.6,
    'rsi_ub': 49.7
}

data = get_backdata('BTCUSD', '1h', 100000)
data = macdas_strat(data, config)

In [ ]:
from time import perf_counter
import numpy as np
import json
class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)
start = perf_counter()
res = quick_backtest(data)
td = perf_counter() - start
print(td, 'seconds')
print('score:', res[1])
print('trade count:', len(res[0]))
print(json.dumps(res[0], indent=2, cls=CustomJSONizer))
from yuzu.strategies.macdas_strat import plot
data[['bought', 'sold', 'stop_lossed', 'stop_loss', 'trade_profit']] = [None, None, None, None, None]
plot(data, 'live').show()

In [ ]:
from yuzu.exchanges.BinanceUS import get_backdata
from yuzu.strategies.macdas_strat import macdas_strat, plot

config = {
    'buy_slow_len': 26,
    'buy_fast_len': 12,
    'buy_sig_len': 9,
    'buy_rsi_len': 24,
    'sell_slow_len': 26,
    'sell_fast_len': 9,
    'sell_sig_len': 6,
    'sell_rsi_len': 14,
    'rsi_lb': 38,
    'rsi_ub': 68,
    'ma_fast_len': 50,
    'ma_slow_len': 20
}

fig = plot(macdas_strat(get_backdata('BTCUSD', '1m', 5000), config, True), config)
fig.update_layout(height=2000)

In [ ]:
def condition_params(**kwargs):
    for k,v in kwargs.items():
        if k == 'a':
            v *= 2
        if k == 'b':
            v += 2
        if k == 'c':
            v -= 2
        print(k, v)
        kwargs[k] = v
    return kwargs.values()

a1, b1, c1 = 1, 2, 3
a1, b1, c1 = condition_params(a=a1, c=c1, b=b1)
print(a1, b1, c1)

In [1]:
from yaza.binance import Exchange
from yuzu.strategies.macdas_strat import macdas_strat
from time import sleep
import json

config_file = open('./config.json')
config = json.loads(config_file.read())['strategies']['macdas_strat']['1m']
trader = Exchange(macdas_strat, config)
trader.start_trading('BTCUSD', '1m')
sleep(3600)
trader.stop_trading('BTCUSD')